In [3]:
import os
import numpy as np
import sys
import torch
import pandas as pd

NameError: name '_C' is not defined

In [ ]:

df_dataset = pd.read_json (r'all_data\cardiacSA_withOmmisions.json')
df_extract = df_dataset[['ID_XNAT','joint_self','frames']]

In [ ]:
df_extract

In [ ]:
# writing down all landmarks to a text file.
#be sure you know the what code is doing before you try it.

for i in range(df_extract.shape[0]):
    filename = 'C:/Users/Alvin/Desktop/dissertation/landmark_touse/'+ str(df_extract['ID_XNAT'][i]) + '.txt'
    f = open(filename, 'a')
    for item in df_extract['joint_self'][i]:
        f.writelines([str(item[0]),' ',str(item[1]),' ',str(item[2]),'\n'])
f.close()

In [ ]:
# writing down landmarks and center landmarks to a text file
# only used when adding center point
for i in range(df_extract.shape[0]):
    filename = 'C:/Users/Alvin/Desktop/dissertation/landmark_touse/'+ str(df_extract['ID_XNAT'][i]) + '.txt'
    f = open(filename, 'a')
    sum_x = 0
    sum_y = 0
    for item in df_extract['joint_self'][i]:
        sum_x += item[0]
        sum_y += item[1]
        f.writelines([str(item[0]),' ',str(item[1]),' ',str(item[2]),'\n'])
    sum_x = sum_x//3
    sum_y = sum_y//3
    f.writelines([str(sum_x),' ',str(sum_y),' ',str(item[2]),'\n'])
f.close()

In [ ]:
# split the dataset by writing data and test data id into two text files


train_name = 'C:/Users/Alvin/Desktop/dissertation/data/'+'data_train.txt'
test_name = 'C:/Users/Alvin/Desktop/dissertation/data/'+'data_test.txt'
for i in range(df_extract.shape[0]):
    if i < 280:
        f_1 = open(train_name, 'a')
        f_1.writelines([str(df_extract['ID_XNAT'][i]),'\n'])
    if i >= 280:
        f_2 = open(test_name, 'a')
        f_2.writelines([str(df_extract['ID_XNAT'][i]),'\n'])
        
f_1.close()
f_2.close()

In [ ]:
# copy image to the destination file location

import shutil 
input_path = 'C:/Users/Alvin/Desktop/dissertation/all_data/'
out_path = 'C:/Users/Alvin/Desktop/dissertation/data/data'
df_extract['ID_XNAT'][i]
for i in range(df_extract.shape[0]):
    input_data = 'C:/Users/Alvin/Desktop/dissertation/all_data/'+str(df_extract['frames'][i]['1'])
    out_path = 'C:/Users/Alvin/Desktop/dissertation/data/data/'+str(df_extract['ID_XNAT'][i])+'.png'
    shutil.copy(input_data,out_path)


In [ ]:
from skimage import io
from skimage import transform as sk_transform
from skimage.transform import rescale, resize, downscale_local_mean
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import glob
import torchvision.transforms as transforms
from PIL import Image


# load image as image object
def load_image(img_path):

    im = io.imread(img_path, as_gray=True)
    
    return im

# load image as image with shape of 101*101, downsized
def load_image_101(img_path):
    
    im = io.imread(img_path, as_gray=True)

    image_resized = resize(im,(101,101), anti_aliasing=True)
    
    return image_resized
    
#load image as image object and using mean-std normalization
def image_reshape_to_numpy(img):
    
    img = np.asarray(img)
    
    img = image_normalized(img)
    
    img_arr = img.reshape(img.shape[0], img.shape[1],1).astype('float32')
    
    return img_arr

# load image as image object and using min-max normalization
def image_reshape_to_numpy_v2(img):
    
    img = np.asarray(img)
    
    img = image_normalized_v2(img)
    
    img_arr = img.reshape(img.shape[0], img.shape[1],1).astype('float32')
    
    return img_arr
    
# mean-std normalization
def image_normalized(ndarray):

    arr_mean = np.mean(ndarray)
    arr_std = np.std(ndarray)
    
    result = (ndarray-arr_mean)/arr_std
    
    return result    

#min-max normalization
def image_normalized_v2(ndarray):

    arr_min = np.min(ndarray).astype('float32')
    arr_max = np.max(ndarray).astype('float32')
    
    result = (ndarray.astype('float32')-arr_min)/(arr_max-arr_min)
    
    return result    



In [ ]:

img_path = 'C:/Users/Alvin/Desktop/dissertation/data/data/PHD_78.png'

img_ = load_image(img_path)

norm = image_normalized_v2(img_)

plt.figure(figsize=(10,5))
plt.subplot(2,2,1)
plt.title("Before Normalisation")
plt.plot(img_)

plt.subplot(2,2,2)
plt.title("After Normalisation")
plt.plot(norm)

plt.subplot(2,2,3)
plt.imshow(img_)

plt.subplot(2,2,4)
plt.imshow(norm)

plt.show()

In [ ]:

def extract_landmark_3D(filepath):

    f= open(filepath, 'r')
    line = f.readlines()
    landmark_list = []
    for l in line:
        row = l.split(' ')
        row[0], row[1], row[2] = row[0].replace('\'',''), row[1].replace('\'',''),row[2].replace('\'','')
        landmark_list.append([int(float(row[0])),int(float(row[1])),int(float(row[2]))])
    f.close()
        
    return landmark_list



In [ ]:
import matplotlib.pyplot as plt
from skimage import io
from skimage import transform as sk_transform
from skimage.transform import rescale, resize, downscale_local_mean
import torch
import glob

test_path = 'C:/Users/Alvin/Desktop/dissertation/result/model2_train_290_4/landmarks/test/'
gt_path = 'C:/Users/Alvin/Desktop/dissertation/Landmark_detection_PIN_center/landmark_detection_PIN/data_2d/landmarks/'
img_file = 'PHD_78_ps.txt'
gt_file = 'PHD_78.txt'
img_path = 'C:/Users/Alvin/Desktop/dissertation/data/data/PHD_78.png'


# load image as image object
def load_image(img_path):

    im = io.imread(img_path, as_gray=True)
    
    return im

# print all landmarks
def show_landmarks(image, landmarks,color):
    """Show image with landmarks"""
    plt.imshow(image)
    for i in range(len(landmarks)):
        plt.scatter(landmarks[i][0],landmarks[i][1],s=100,c=color)

# print landmarks with center point landmark
def show_landmarks_center(image, landmarks,color):
    """Show image with landmarks"""
    plt.imshow(image)
    for i in range(len(landmarks)):
        plt.scatter(landmarks[i][0],landmarks[i][1],s=100,c=color)
        plt.scatter(landmarks[len(landmarks)-1][0],landmarks[len(landmarks)-1][1],s=100,c='c')

# print landmarks excluding the center point
def show_landmarks_test(image, landmarks,color):
    """Show image with landmarks"""
    plt.imshow(image)
    for i in range(len(landmarks)-1):
        plt.scatter(landmarks[i][0],landmarks[i][1],s=100,c=color)

def extract_landmark_2d(filepath):

    f= open(filepath, 'r')
    line = f.readlines()
    landmark_list = []
    for l in line:
        row = l.split(' ')
        row[0], row[1] = row[0].replace('\'',''), row[1].replace('\'','')
        landmark_list.append([int(float(row[0])),int(float(row[1]))])
    f.close()
        
    return landmark_list

landmark_list_true = extract_landmark_2d(gt_path+ gt_file)
landmark_list_pred = extract_landmark_2d(test_path+ img_file)

In [ ]:

plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.title("Ground True")
show_landmarks(load_image(img_path),
               landmark_list_true,'r')

plt.subplot(1,2,2)
plt.title("Prediction")
show_landmarks(load_image(img_path),
               landmark_list_pred,'y')

plt.show()

In [ ]:
# calculate result
test_path = 'C:/Users/Alvin/Desktop/dissertation/result/model2_train_290_4/error.txt'
img_file = 'PHD_6243_ps.txt'

def extract_landmark(filepath):

    f= open(filepath, 'r')
    line = f.readlines()
    landmark_list = []
    for l in line:
        row = l.split(' ')
        row[0], row[1] = row[0].replace('\'',''), row[1].replace('\'','')
        landmark_list.append([int(float(row[0])),int(float(row[1]))])
    f.close()
        
    return landmark_list


In [ ]:
# extract the original 3 landmarks for calculation.
# 

f= open(test_path, 'r')
line = f.readlines()
error_list = []
error_list_full = []
error_list_x = []
error_list_y = []
error_list_z = []
for l in line:
    row = l.split(' ')
    row[1], row[2],row[3] = row[1].replace('\'',''), row[2].replace('\'',''),row[3].replace('\'','')
    error_list.append([float(row[1]),float(row[2]),float(row[3])])
    error_list_full.append(float(row[1]))
    error_list_full.append(float(row[2]))
    error_list_full.append(float(row[3]))
    error_list_x.append(float(row[1]))
    error_list_y.append(float(row[2]))
    error_list_z.append(float(row[3]))
f.close()

In [ ]:
# used in center point method
# calculate the std and mean based on results with the center point method applied from the original 3 landmarks
import statistics

def calculate_std_mean(input_list):
    std_result = statistics.stdev(input_list)
    mean_result = statistics.mean(input_list)
    
    return std_result,mean_result

In [ ]:
std_,mean_ = calculate_std_mean(error_list_full)
print('std:',std_)
print('mean:',mean_)

std_,mean_ = calculate_std_mean(error_list_x)
print('std:',std_)
print('mean:',mean_)

std_,mean_ = calculate_std_mean(error_list_y)
print('std:',std_)
print('mean:',mean_)

std_,mean_ = calculate_std_mean(error_list_z)
print('std:',std_)
print('mean:',mean_)

In [ ]:
import glob
import re

test_file = glob.glob(r'C:/Users/Alvin/Desktop/dissertation/result/model2_train_290_4/landmarks/test/*.txt')
gt_path = 'C:/Users/Alvin/Desktop/dissertation/Landmark_detection_PIN_center/landmark_detection_PIN/data_2d/landmarks_4/'
img_path = 'C:/Users/Alvin/Desktop/dissertation/data/data/'

test_print =[]
gt_print = []
img_print = []
for i in test_file:
    a = re.split('\\\\', i)
    b = a[-1].replace('_ps','')
    c = b.replace('.txt','.png')
    test_print.append(a[-1])
    gt_print.append(b)
    img_print.append(c)

def print_results(img_gt,img_test,img_name):
    landmark_list_true = extract_landmark_2d(gt_path+ img_gt)
    landmark_list_pred = extract_landmark_2d(test_path+ img_test)
    
    plt.figure(figsize=(15,15))
    plt.subplot(1,2,1)
    plt.title("Ground True")
    show_landmarks_center(load_image(img_path+img_name),
                   landmark_list_true,'r')

    plt.subplot(1,2,2)
    plt.title("Prediction")
    show_landmarks_test(load_image(img_path+img_name),
                   landmark_list_pred,'y')

    plt.show()

for i in range(len(test_print)):
    print_results(gt_print[i],test_print[i],img_print[i])